In [1]:
from vllm import LLM
import torch

In [2]:
loaded_directions = torch.load('directions.pth')


In [3]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1", enable_cv=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-21 08:46:52 llm_engine.py:163] Initializing an LLM engine (v0.5.0.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.1', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=mistralai/Mistral-7B-Instruct-v0.1)
INFO 06-21 08:46:53 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-21 08:46:53 selector.py:51] Using XFormers backend.
INFO 06-21 08:46:57 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
I

In [ ]:
from vllm.control_vectors.request import ControlVectorRequest
from vllm.control_vectors.control import ControlVector


cv = ControlVector(model_type="mistral", directions=loaded_directions)

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=100,
    stop=["[/assistant]"]
)

In [ ]:
prompts = "what are apples"

In [ ]:
outputs = llm.generate(
    prompts,
    sampling_params,
    control_vector_request=ControlVectorRequest("sql_adapter", cv, coefficient=0.01)
)



In [ ]:
outputs